In [46]:
import sys
import os
import pandas as pd
sys.path.append("/Users/natwat/Desktop/CPSC_Projects/InvBankAI/")
PROJECT_ROOT = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/"

In [47]:
%run /Users/natwat/Desktop/CPSC_Projects/INVBANKAI/scripts/FH_getSent.py

2025-07-14 15:11:24,120 [INFO] Using device: mps
Device set to use mps:0
2025-07-14 15:11:34,367 [INFO] Fetched 5 articles


In [48]:
%run /Users/natwat/Desktop/CPSC_Projects/INVBANKAI/scripts/dowload_data.py
%run /Users/natwat/Desktop/CPSC_Projects/INVBANKAI/scripts/compute_ema.py  ##---> This gives features.csv 


Saved raw data for TSLA → /Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/raw/TSLA_raw.csv
Saved features for TSLA → /Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/features/TSLA_features.csv


/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/scripts/compute_ema.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(
/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/scripts/compute_ema.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ema_col] = df['Close'].ewm(span=period, adjust=False).mean()


In [ ]:
import sys
import os

# Add project root to path for config import
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import get_ticker, get_paths

from combineSingleDB import load_and_aggregate_sentiment, full_incremental_merge, full_merge

# Get paths from config
PATHS = get_paths()
sent_path = PATHS["sentiment_combined"]
price_path = PATHS["features_csv"]  # Must be minute-level
output_path = PATHS["features_full_csv"]
archive_path = PATHS["archive_csv"]

# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(sent_path, resample_interval="1min")
sentiment_df.tail()

In [50]:

from combineSingleDB import full_merge
# Get last timestamp in existing merged file
if os.path.exists(price_path):
    df = pd.read_csv(price_path, parse_dates=["Datetime"])
    last_ts = df["Datetime"].max()
    
else:
    last_ts = None

# Run the updated merge
full_merge(sentiment_df, price_path, output_path)
# View output
df = pd.read_csv(output_path)
df.tail()

✅ Saved merged data to /Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label
2317,2025-07-14 19:55:00+00:00,316.790009,317.000000,316.550110,316.750000,428902.0,316.435484,1.695669e-07,5.004997e-08,1.0,0.080077,0.910761,0.009162,0.99339,0.0,0.00661,0.357822,0.551872,-0.000316,0
2318,2025-07-14 19:56:00+00:00,316.690002,316.929901,316.589996,316.779999,260013.0,316.459724,1.695669e-07,5.004997e-08,1.0,0.080077,0.910761,0.009162,0.99339,0.0,0.00661,0.357822,0.551872,0.000469,1
2319,2025-07-14 19:57:00+00:00,316.838593,316.890015,316.690002,316.700012,284435.0,316.495807,1.695669e-07,5.004997e-08,1.0,0.080077,0.910761,0.009162,0.99339,0.0,0.00661,0.357822,0.551872,0.000762,1
2320,2025-07-14 19:58:00+00:00,317.079987,317.109985,316.820007,316.829987,486575.0,316.551443,1.695669e-07,5.004997e-08,1.0,0.080077,0.910761,0.009162,0.99339,0.0,0.00661,0.357822,0.551872,-0.000536,0
2321,2025-07-14 19:59:00+00:00,316.910004,317.269989,316.809998,317.079987,666850.0,316.585592,1.695669e-07,5.004997e-08,1.0,0.080077,0.910761,0.009162,0.99339,0.0,0.00661,0.357822,0.551872,NaN,0


In [ ]:
import ta

# Use path from config
df = pd.read_csv(PATHS["features_full_csv"], parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  
df.tail()

In [ ]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]

# Use path from config
df.to_csv(PATHS["ready_csv"], index=False)
df.tail()

In [53]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [55]:
arch_df = pd.concat(df, pd.read_csv(archive_path, parse_dates = ["Datetime"]), ignore_index=True)

arch_df.tail()

TypeError: concat() got an unexpected keyword argument 'parse_dates'